# Cvičenie 7: Q-Learning a SARSA

Na dnešnom cvičení sa budeme zaoberať algoritmami typu temporal difference, a to konkrétne Q-učením a algoritmom SARSA. Algoritmy budeme aplikovať na riešenie už známeho prostredia Gridworld.

## Q-učenie

Q-učenie (*Q-Learning*), podobne ako Monte Carlo metódy, sa učí priamo z interakcií s prostredím, jeho priebeh teda závisí od priebehu konkrétnych epizód. Q-Learning zároveň nevyžaduje známy model prostredia, prechody medzi stavmi sa buď nasimulujú, alebo určia zo skutočnej interakcie agenta s prostredím. Algoritmus zvyčajne pracuje s $\varepsilon$-greedy politikou, kde s najväčšou pravdepodobnosťou sa vyberie akcia, ktorú agent vníma ako najlepšiu možnú, pritom ale existuje možnosť na výber náhodnej akcie. Q-learning je ale off-policy metóda, to znamená, že pri aktualizácii odhadu hodnôt dvojice stav-akcia sa neberie do úvahy práve platná politika agenta.

Pseudokód nájdete na obrázku nižšie, prediskutujte jeho kroky a význam premenných a parametrov, s ktorými algoritmus pracuje.

<img src="sources/lab07/q-learning.jpg">

Následne algoritmus aplikujeme pri riešení už známeho príkladu gridworld:

<img src="sources/lab07/gridworld_example2.jpg" width="300">

Príklad predstavuje svet *3x3* s cieľovou pozíciou v pravom hornom rohu, a s jednou pascou v druhom riadku a prvom stĺpci. K dispozícii sú štyri akcie: posun na sever, východ, juh a západ. Hráč nemôže prejsť cez stenu, v tomto prípade ostane na rovnakej pozícii, ale berie sa, akokeby vykonal krok. Ak sa hráč dostane do cieľa, obdrží odmenu 10, ak spadne do pasce, tak -10. V oboch prípadoch sa hra skončí. Pre ostatné kroky dostane agent odmenu -1.

Discount factor $\gamma = 0.8$. Učiaci parameter $\alpha = 0.2$; $\varepsilon = 0.1$. Prvý odhad hodnoty pre každú dvojicu stav-akcia bude 0: $Q(s, a) = 0 \ \ \  \forall s \in \mathcal{S}, a \in \mathcal{A}$.

## Ukážka fungovania algoritmu

Priebeh algoritmu si ukážeme na nasledovných epizódach:

$s_{22}, W, -10, s_{21}$

$s_{31}, S, -1, s_{31}, E, -1, s_{32}, W, -1, s_{31}, N, -10, s_{21}$

$s_{12}, S, -1 s_{22}, S, -1, s_{32}, N, -1, s_{22}, N, -1, s_{12}, E, 10, s_{13}$

### 1. epizóda

Prvá epizóda sa skladá iba z jedného kroku, pri ktorom aktualizujeme iba hodnotu $Q(s_{22}, W)$. K tomu okrem získaných údajov potrebujeme len vedieť maximálnu Q-hodnotu pre nasledujúci stav $s_{21}$. Keďže sa jedná o terminálny stav, platí:

$\underset{a}{max} Q(s_{21}, a) = 0.0$

A teda pri aktualizácii platí:

$Q(s_{22}, W) \leftarrow Q(s_{22}, W) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{21}, a) - Q(s_{22}, W))$

$Q(s_{22}, W) \leftarrow 0.0 + 0.2 \cdot (-10 + 0.8 \cdot 0.0 - 0.0)$

$Q(s_{22}, W) \leftarrow -2$

### 2. epizóda

Pri Q-learningu vykonáme aktualizáciu hodnôt pre každý krok, teda pre druhú epizódu platia rovnaké výpočty:

$Q(s_{31}, S) \leftarrow Q(s_{31}, S) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{31}, a) - Q(s_{31}, S))$

$Q(s_{31}, S) \leftarrow 0.0 + 0.2 \cdot (-1 + 0.8 \cdot 0.0 - 0.0)$

$Q(s_{31}, S) \leftarrow -0.2$

Pre druhý krok:

$Q(s_{31}, E) \leftarrow Q(s_{31}, E) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{32}, a) - Q(s_{31}, E))$

$Q(s_{31}, E) \leftarrow 0.0 + 0.2 \cdot (-1 + 0.8 \cdot 0.0 - 0.0)$

$Q(s_{31}, E) \leftarrow -0.2$

Pre tretí krok:

$Q(s_{32}, W) \leftarrow Q(s_{32}, W) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{31}, a) - Q(s_{32}, W))$

$Q(s_{32}, W) \leftarrow 0.0 + 0.2 \cdot (-1 + 0.8 \cdot 0.0 - 0.0)$

$Q(s_{32}, W) \leftarrow -0.2$

A pre posledný krok:

$Q(s_{31}, N) \leftarrow Q(s_{31}, N) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{21}, a) - Q(s_{31}, N))$

$Q(s_{31}, N) \leftarrow 0.0 + 0.2 \cdot (-10 + 0.8 \cdot 0.0 - 0.0)$

$Q(s_{31}, N) \leftarrow 2$

### 3. epizóda

Spracujte 3. ukážkovú epizódu a skontrolujte získané Q-hodnoty podľa tabuľky:

| stav |   N  |   E  |   S  |   W  |
|:----:|:----:|:----:|:----:|:----:|
| s11  |    0 |    0 |    0 |    0 |
| s12  |    0 |    2 | -0.2 |    0 |
| s13  |    0 |    0 |    0 |    0 |
| s21  |    0 |    0 |    0 |    0 |
| s22  | -0.2 |    0 | -0.2 |   -2 |
| s23  |    0 |    0 |    0 |    0 |
| s31  |   -2 | -0.2 | -0.2 |    0 |
| s32  | -0.2 |    0 |    0 | -0.2 |
| s33  |    0 |    0 |    0 |    0 |

Ďalší priebeh algoritmu si môžete vizualizovať pomocou [ukážkového kódu](sources/lab07/q-learning.zip). Porovnajte vypočítané Q-hodnoty s riešením Bellmanových rovníc optimality.

## SARSA

On-policy variantom algoritmu Q-Learning je SARSA, ktorý oproti Q-Learningu berie do úvahy aj nasledujúcu akciu pri aktualizácii Q-hodnoty. Prediskutujte jednotlivé kroky algoritmu a porovnajte ich s krokmi algoritmu Q-Learning.

<img src="sources/lab07/sarsa.jpg">

Následne si stiahnite [implementáciu algoritmu SARSA](sources/lab07/sarsa.py), ktorá ale obsahuje rovnaký kód, ako Q-learning. Opravte kód tak, aby implementoval algoritmus SARSA.

## Domáca úloha

Pri aplikácii algoritmov Q-Learning a SARSA sa často používa takzvaný $\varepsilon$-decay, kde na začiatku trénovania algoritmus má vysokú hodnotu $\varepsilon$ (často $1$), ktorá sa časom zmenšuje tak, že sa vynásobí konštantou $decay < 1$. Podobným štýlom sa dá prispôsobovať aj učiaci parameter počas trénovania. Rozšírte implementáciu Q-Learningu (alebo SARSA) o takýto decay, a porovnajte mieru konvergencie na danej mape. Výsledky porovnania napíšte do komentárov. Riešenie odovzdávate cez MS Teams.